<center><span style="font-size:24px;"><b> Physical Activity Recognition Group 15 - round 1 </b></span></center>


<h1 id=introduction><br><span style="font-size: 20px;"><b> Introduction to the project </b> </span></h1>

<span style="font-size: 16px;"> In this notebook we build a classifier algorithm that recognizes different types of physical movement and static postures (e.g. walking, sitting, standing up) based upon measurements from a smartphone accelerometer and a gyroscope. To train algorithm we use data provided the competition hosts Daniel van der Meer, Joost van Kordelaar, Dave Leitritz and Raoul Grasman (2023). This data was collected during experiments where a group of 30 participants had to complete a sequence of several movements and postures and measurements were taken by a smartphone strapped to their body from beginning to end of the sequence. This includes transitions periods such as sitting to standing. The accelerometer and a gyroscope data comes in the from of a constant rate of 50Hz measurments of three different axes.</span>
    
<span style="font-size: 16px;"> Because one sequence of measurments contains multiple movements and postures, the total sequence is divided up in sections of sampled epochs, each of which has to be labeld with a movement or posture. This is the eventual output of the algorithm. In this notebook we show our way of tackling this problem, from loading in the data, to feature selection, to model creation and selection and finnaly arrive with a set of predictions which we submit as part of the kaggle competition. </span>

<span style="font-size: 20px;"><b> Structure of the notebook </b> </span>
* R-setup
* Importing data
* Pre-processing data
* Feature creation
* Model creation
* Model selection
* Model prediction

<h1 id=rsetup><span style="font-size: 20px;"><b> Setting up R </b> </span></h1>

We load in packages, turn off warnings, and copy all files to our working directory

In [ ]:
# load in packages
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(caret))


# handle warnings
options(warn=-1)    # warnings off
# options(warn=0)   # warnings on

.... = NA

# Copy all files to the working directory
system(paste0("cp -r ", list.files("../input", pattern = "recognition", full.names=TRUE), "/* ./"))

<h1><span style="font-size: 20px;"><b>  Reading in the labels data </b></span></h1>

In this section we will import and structure data related to the activity labels (dependent variables):

* The dependent variables ential physical activities, encompassing 12 labels. These labels represent activities such as walking, lying down, standing, and sitting. Some labels also capture transitions between these states, like moving from standing to sitting. Furthermore, we differentiate between walking downstairs and upstairs.

* The goal is to develop a predictive model accurately forecasting these physical activity labels. To achieve this, we will leverage a collection of labeled files, which serve as our training data. Subsequently, we will assess the model's generalization ability by applying it to a set of unlabeled test data. We will evaluate its performance by constructing a confusion matrix, which will help us compare our predicted labels with the actual labels from the test data.

<br><span style="font-size: 16px;"><b> We import activity labels and convert numeric activity labels into lettered labels. </b></span>

In [ ]:
# load in activity lables
act_labels <- read_delim("activity_labels.txt",
                         " ",
                         col_names = FALSE, 
                         trim_ws = TRUE,
                         show_col_types = FALSE) 

act_labels <- act_labels %>% select(X1, X2)

# load in labels
labels <- read_delim("./RawData/Train/labels_train.txt",
                     " ",
                     col_names = FALSE,
                     show_col_types = FALSE)
colnames(labels) <- c('trial', 'userid', 'activity', 'start', 'end')

# swap out number label for word label
labels <- labels %>% mutate(activity = act_labels$X2[activity])

head(labels)

<span style="font-size: 16px;"><b> We add a new column of lists that contain a sequence of sample id's. </b></span>
    
A sampleID essentially a description of a moment of measurement, or a place in the sequence of measurements. The lists we make in the next cell contain all the sampleIDs from the start of an activity label to the end of the activity label. In the print statement you can see the length of the lists, or rather the number of sampleIDs they contain.

In [ ]:
# Add the sequence start:end to each row in a list.
# The result is a nested table:
sample_labels_nested <-
    labels %>% 
    rowwise() %>% # do next operation(s) rowwise
    mutate(sampleid = list(start:end)) %>%
    ungroup()

print(sample_labels_nested)

<span style="font-size: 16px;"><b> We unnest the lists of sample id's </b></span>

Lists that are nested into a table are complicated to access. Therefore we unnest the lists into one big table we can easily use later.

In [ ]:
# Unnest the nested tabel
sample_labels <-
    sample_labels_nested %>% 

    # Rows are segments, we need to keep track of different segements
    mutate(segment = row_number() ) %>% 

    # Expand the data frame to one sample per row
    unnest(cols = c(sampleid)) %>% 

    # Remove columns we don't need anymore
    select(-start, -end)

print(sample_labels)

<span style="font-size: 16px;"><b> The final labels table </b></span>

The labels table we created contains alot important information about our measurements. We have for each measurement, what trial was going on, who was being measured, what activity they were doing, at what point in the sequence of samples it came and the segment in which it came.
_______________________________________________________________________________________________________

________________________________________________________________________________________________________
<h1 id=functions><span style="font-size: 20px;"><b> Feature extraction functions </b></span></h1>

Before we start extracting features from the data signals we define helper functions to help keep the code clean and simple. We definde two types of helper functions those related to time domain features and time frequency features. The difference between them is that time domain features relate to the actual values of the measurements, while time frequency features relate to spectral properties.

The data processing approach involved organizing the sample IDs into epochs, with each epoch containing 128 samples. Within each epoch, we extracted features from the data. The activity label assigned to each epoch was determined by identifying the most frequently occurring activity during that specific epoch.

<span style="font-size: 16px;"><b> Time domain features </b></span>

*Description of created time domain features*

* **Most common value**: Returns the value that has the highest occurrence count within a given set of values (it is also known as the mode in statistics).

* **Lagged correlations**: It measures the correlation between two time series variables with a time delay or lag between them. In other words, lagged correlations explore how one variable is related to another when there is a temporal offset between their observations (Grasman, 2018).

* **Root mean square (RMS)**: It is used to find the average value of a set of values or the "typical" magnitude of a varying quantity. The RMS is particularly useful when dealing with values that fluctuate, such as time-varying signals or data. First we have to square the values, calculate their mean and take their square root.

* **Scaled inner product (SIP)**: It measures two vectors' similarity in space, and scaling involves multiplying this similarity measure by a constant factor (Grasman, 2018).

In [ ]:
## Feature functions

# Most common value (base)
most_common_value = function(x) {
    counts = table(x, useNA='no')
    most_frequent = which.max(counts)
    return(names(most_frequent))
}

# Lagged correlations (base)
lagged_cor <- function(x, y=x, lag=0) {
    # compute correlation between x and a time shifted y
    r_lagged = cor(x, dplyr::lag(y, lag), use='pairwise')
    r_lagged = ifelse (is.na(r_lagged), 0, r_lagged)
    return(r_lagged)
}

# Root mean square (internet)
rms <- function(x){
  sqrt((1/length(x)*(sum(x^2))))
}

# Scaled inner product (Grasman, R. (2018))
SIP = function(x1, x2) {
    sip = x1 %*% x2 / sqrt(sum(x1^2) * sum(x2^2))
    sip = as.double(sip)
    return(sip)
}

<span style="font-size: 16px;"><b> Frequency domain features </b></span>

*Description of created frequency domain features*

* **Entropy**: It measures the amount of uncertainty in a set of data or a probability distribution, therefore quantifies the unpredictability of the outcomes in a dataset. Overall, it returns the amount of information present (Grasman, 2018). 

* **Energy**: Returns the “total power” of a signal in n samples (Grasman, 2018).

* **Spectral peak**: Returns a distinct local maximum in the spectral density or power spectrum of a signal. The spectral density or power spectrum is a representation of how the power or energy of a signal is distributed across different frequencies.

* **Spectral entropy**: Returns the etropy of a spectrum, quantifying the amount of spectral information. It measures how uniformly the power or energy of a signal is distributed across different frequency components (randomness of frequency components in a signal) (Grasman, 2018).

* **Spectral mean and standard deviation (SD)**: Mean and SD of a spectrum (Grasman, 2018). It summarizes the characteristics of a signal's frequency spectrum.

* **Spectral mode, median, kurtosis (from Group 16)**: Most common spectral frequency, median spectral frequency and the sharpness of the spectral distribution.

In [ ]:
# Entropy (Grasman, R. (2018))
entropy  <- function(x, nbreaks = nclass.Sturges(x)) {
    r = range(x)
    x_binned = findInterval(x, seq(r[1], r[2], len = nbreaks))
    h = tabulate(x_binned, nbins = nbreaks)
    p = h/sum(h)
    entropy = -sum(p[p>0] * log(p[p>0]))
    return(entropy)
}

# Energy (Grasman, R. (2018))
energy = function(x) {
  sum(x^2)
}

# Spectral peak (Grasman, R. (2018))
spec_peak = function(x) {
  spec = spectrum(x, plot = FALSE)
  return(spec$freq[which.max(spec$spec)])
}

# Spectral entropy (Grasman, R. (2018))
spec_entr = function(x) {
    spec = spectrum(x, plot = FALSE)$spec
    entropy(spec)
}

# Spectral mean (Grasman, R. (2018))
spec_avg <- function(x) {
  spec = spectrum(x, plot = F)
  df = spec$freq[2] - spec$freq[1]
  return(sum(spec$freq * spec$spec * df))
}

# Spectral standard deviation (Grasman, R. (2018))
spec_sd <- function(x) {
    spec = spectrum(x, log = 'n', plot = FALSE)$spec
    freq = spectrum(x, log = 'n', plot = FALSE)$freq
    df = freq[2] - freq[1]
    
    return(sqrt(sum((freq - mean(x))^2 * spec * df)))
}

# Spectral Mode, Median, and Kurtosis
# in analogy from Grasman (2018) but taken from Group 16
spec_mode <- function(x) {
    spec_features <- spectrum(x, plot = FALSE)
    mode <- max(spec_features$spec)
    return(mode)
}

spec_median <- function(x) {
    spec_features <- spectrum(x, plot = FALSE)
    median <- median(spec_features$spec)
    return(median)
}

spec_kurt <- function(x) {
    spec_features <- spectrum(x, plot = FALSE)
    kurt <- e1071::kurtosis(spec_features$spec)
    return(kurt)
}

<b> Stolen features </b>

Because all good artists steal, we looked at some submissions from previous iterations of this competition. Michael Ogwuru, Wesley Korff and 
Elisa Mens (group 12) were the winners of last years competition came up with a feature that compares the mean of intervals within an epoch to that of the overal variable. For this they used the following helper function.

We also take the idea of using derivatives from Daphne Jonkers Both and Noah Hatakeyama (last years group 10)

With the use of derivates they compute twelve total features:
- mean, sd, skew, and kurtosis of linear jerk
- mean, sd, skew, and kurtosis of angular acceleration
- mean, sd, skew, and kurtosis of angular jerk

We won't use the last bit, since that would require significant changes to our code.

In [ ]:
# from last years group 12

# The difference in mean of epoch with overall mean 
average_within <- function(x){
    interval <- as.numeric()
    interval <- c(x[1:9] %>% mean(),
                  x[10:19] %>% mean(),
                  x[20:29] %>% mean(),
                  x[30:39] %>% mean(),
                  x[40:49] %>% mean(),
                  x[50:59] %>% mean(),
                  x[60:69] %>% mean(),
                  x[70:79] %>% mean(),
                  x[80:89] %>% mean(),
                  x[90:99] %>% mean(),
                  x[100:109] %>% mean(),
                  x[110:119] %>% mean(),
                  x[120:128] %>% mean())
    average_interval <- as.numeric()
    for (i in 1:13){
        average_interval[i] <- interval[i] - mean(x)
    }
    return(sum(average_interval))
}

# from last years group 10
# Computes first derivative, applies the function specified in `type` and returns it
single_deriv <- function(x, type=mean, ...) {
    x %>% 
    diff %>% 
    type(., ...) %>% 
    return
}

________________________________________________________________________________________________________

________________________________________________________________________________________________________
<h1 id=combi><span style="font-size: 20px;"><b> Combining the feature extration functions </b></span></h1>

Now that we have put in place all the helper functions that we need we can define our big feature extraction function. It's a big function with several steps to it. It's going to take as input a file name and the sample labels table that we created previously. From the file name it's going extract the username and trial id and import the sensor signals from the file. We then create a column which ranks the order of appaerance of the sensor measurements. This is the sampleID we also have stored in labels. We now have the required information to join in the label activity data from our sample_labels table.

Next up our function is going split our data into epochs of 128 sample and compute our features per epoch. One epoch is a timeframe of 128 samples, which corresponds to 2.56 seconds.

We extract the following list of features for each spatial axis (remember that the gyroscope and accellorater record data on three axes).

- mean 
- most common values 
- median
- range
- 25th quantile
- 75th quantile
- skewness
- kurtosis
- power
- root mean square
- number of values below mean
- entropy
- energy
- number of positive features
- spectral peak
- spectral energy
- spectral mean
- spectral standard deviation
- spectral standard mode
- spectral standard median
- spectral standard kurtosis
- auto_correlation with lags 1 and 2

Furthermore several features are extracted based on the relationship between the axes

- lagged_correlations with lags 1 and 2 between all features
- mean difference between axes
- scaled inner product

Lastly we steal some features from groups that competed in lasts competition
We steal from group 1 (Pander Abbing, Meike Waaijers and Paulo Ortiz) and from group 12 (Michael Ogwuru, Wesley Korff and 
Elisa Mens)

- signal magnitude area
- width to height ratio
- average resultant acceleration
- difference in mean of intervals within epoch with overall mean

We also compute the most common activity label in each epoch, keep track of userid, experimentid and compute the number of samples.



At the end of the function it returns the table with all the features it extracted.

In [ ]:
extractDomainFeatures <- function(filename, sample_labels) {
    
    # extract user and experimental run ID's from file name
    username = gsub(".+user(\\d+).+", "\\1", filename) %>% as.numeric()
    expname  = gsub( ".+exp(\\d+).+", "\\1", filename) %>% as.numeric()
    
    # import the sensor signals from the file
    user01 <- read_delim(filename, " ", col_names = F, progress = TRUE, 
                 col_types = "ddd")
    
    # merge signals with labels 
    user_df <- 
        data.frame(userid = username, trial = expname, user01) %>%
        mutate(sampleid = 0:(nrow(user01)-1) ) %>%
        left_join(sample_labels, by = c('userid','trial','sampleid')) 

    
    # split in epochs of 128 samples and compute features per epoch
    usertimedom <-  user_df %>%
    
          # add an epoch ID variable (on epoch = 2.56 sec)
          mutate(epoch = sampleid %/% 128) %>% 

          # extract statistical features from each epoch
          group_by(epoch) %>%
          summarise(
              
            # keep track of user and experiment information
            user_id = username, 
            exp_id = expname,   
              
            # epoch's activity labels and start sample
            activity = most_common_value(c("-", activity)),
            sampleid = sampleid[1],
              
            # features
            # means
            m1 = mean(X1), 
            m2 = mean(X2),
            m3 = mean(X3),
              
            # most common values
            most_common_value1 = as.numeric(most_common_value(X1)),
            most_common_value2 = as.numeric(most_common_value(X2)),
            most_common_value3 = as.numeric(most_common_value(X3)),

            # medians
            median1 = median(X1),
            median2 = median(X2),
            median3 = median(X3),

            # ranges
            range1 = max(X1) - min (X1),
            range2 = max(X2) - min (X2),
            range3 = max(X3) - min (X3),

            # standard deviations
            sd1 = sd(X1),
            sd2 = sd(X2),
            sd3 = sd(X3),

            # quantiles (25% & 75%)
            q1_25 = quantile(X1, .25),
            q2_25 = quantile(X2, .25),
            q3_25 = quantile(X3, .25),
            q1_75 = quantile(X1, .75),
            q2_75 = quantile(X2, .75),
            q3_75 = quantile(X3, .75),

            # skewness
            skew1 = e1071::skewness(X1),
            skew2 = e1071::skewness(X2),
            skew3 = e1071::skewness(X3),

            # kurtosis
            kurt1 = e1071::kurtosis(X1),
            kurt2 = e1071::kurtosis(X2),
            kurt3 = e1071::kurtosis(X3),

            # correlations
            cor12 = cor(X1, X2),
            cor13 = cor(X1, X3), 
            cor23 = cor(X2, X3),  
            
            # lagged correlations
            AR1_1 = lagged_cor(X1, lag=1),
            AR1_2 = lagged_cor(X1, lag=2),
            AR2_1 = lagged_cor(X2, lag=1),
            AR2_2 = lagged_cor(X2, lag=2), 
            AR3_1 = lagged_cor(X3, lag=1),
            AR3_2 = lagged_cor(X3, lag=2),
            AR12_1 = lagged_cor(X1, X2, lag=1),
            AR12_2 = lagged_cor(X1, X2, lag=2),
            AR13_1 = lagged_cor(X1, X3, lag=1),
            AR13_2 = lagged_cor(X1, X3, lag=2),
            AR23_1 = lagged_cor(X2, X3, lag=1),
            AR23_2 = lagged_cor(X2, X3, lag=2),
            
            # power
            power_1 = mean(X1^2),
            power_2 = mean(X2^2),
            power_3 = mean(X3^2),
            
            # root mean square
            rms1 = rms(X1),
            rms2 = rms(X2),
            rms3 = rms(X3),
              
            # mean differences between axes
            mdif1_2 = m1 - m2, 
            mdif1_3 = m1 - m3,
            mdif2_3 = m2 - m3,
              
            # number of values below mean
            nVBM_1 = sum(X1 < mean(X1)),
            nVBM_2 = sum(X2 < mean(X2)),
            nVBM_3 = sum(X3 < mean(X3)),
              
            # scaled inner product
            SIP12 = SIP(X1, X2),
            SIP13 = SIP(X1, X3),
            SIP23 = SIP(X2, X3),
              
            # entropy
            entropy_1 = entropy(X1),
            entropy_2 = entropy(X2),
            entropy_3 = entropy(X3),
              
            # energy
            energy_1 = energy(X1),
            energy_2 = energy(X2),
            energy_3 = energy(X3),
              
            # n of positive features
            NPF_1 = sum(X1 > 0),
            NPF_2 = sum(X2 > 0),
            NPF_3 = sum(X3 > 0),
              
            # spectral peak
            spec_peak_1 = spec_peak(X1),
            spec_peak_2 = spec_peak(X2),
            spec_peak_3 = spec_peak(X3),
              
            # spectral entropy
            spec_entr_1 = spec_entr(X1),
            spec_entr_2 = spec_entr(X2),
            spec_entr_3 = spec_entr(X3),
              
            # spectral mean
            spec_avg_1 = spec_avg(X1),
            spec_avg_2 = spec_avg(X2),
            spec_avg_3 = spec_avg(X3),
              
            # spectral standard deviation
            spec_sd_1 = spec_sd(X1),
            spec_sd_2 = spec_sd(X2),
            spec_sd_3 = spec_sd(X3),
              
            # Spectral Mode
            spec_mode_1 = spec_mode(X1),
            spec_mode_2 = spec_mode(X2),
            spec_mode_3 = spec_mode(X3),
            
            # Spectral Median
            spec_median_1 = spec_median(X1),
            spec_median_2 = spec_median(X2),
            spec_median_3 = spec_median(X3),
            
            # Spectral Kurtosis
            spec_kurt_1 = spec_kurt(X1),
            spec_kurt_2 = spec_kurt(X2),
            spec_kurt_3 = spec_kurt(X3),
            
            #average resultant acceleration (from last years group 1)
            ARA = mean(sqrt(X1^2 + X2^2 + X3^2)),
              
            # signal magnitude area (from last years group 1)
            SMA = sum(abs(X1) / 128) + sum(abs(X2) / 128) + sum(abs(X3) / 128),
              
            # Width to height ratio (from last years group 1)
            whratio1 = X1[which.max(X1)] / length(sampleid),
            whratio2 = X2[which.max(X2)] / length(sampleid),
            whratio3 = X3[which.max(X3)] / length(sampleid),
            
            # Average comparison (from last years group 12)
            average_within_x1 = average_within(X1),  
            average_within_x2 = average_within(X2),
            average_within_x3 = average_within(X3),
              
            # Derivative features # (from last years group 10)
            deriv_mean_X1 = single_deriv(X1),
            deriv_mean_X2 = single_deriv(X2),
            deriv_mean_X3 = single_deriv(X3),
            deriv_sd_X1 = single_deriv(X1, sd),
            deriv_sd_X2 = single_deriv(X2, sd),
            deriv_sd_X3 = single_deriv(X3, sd),
            deriv_skew_X1 = single_deriv(X1, e1071::skewness),
            deriv_skew_X2 = single_deriv(X2, e1071::skewness),
            deriv_skew_X3 = single_deriv(X3, e1071::skewness),
            deriv_kurt_X1 = single_deriv(X1, e1071::kurtosis),
            deriv_kurt_X2 = single_deriv(X2, e1071::kurtosis),
            deriv_kurt_X3 = single_deriv(X3, e1071::kurtosis),
              
            n_samples = n()  
          ) 
    
    usertimedom 
}

<span style="font-size: 16px;"><b> Checking if the feature extration extraction function works </b></span>

Before we continue on with extracting features for all participants. We first want to test our function on the data of the first participant to see if it works.

In [ ]:
# Check if function works for acceleration data
filename <- "./RawData/Train/acc_exp01_user01.txt"
extractDomainFeatures(filename, sample_labels) %>% print()

In [ ]:
# Check if function works for gyro data
filename <- "./RawData/Train/gyro_exp01_user01.txt"
extractDomainFeatures(filename, sample_labels) %>% print()

________________________________________________________________________________________________________

________________________________________________________________________________________________________
<h1 id=features><span style="font-size: 20px;"><b> Extracting the features </h1></span></b>

Now its time to put our function to use and extract features for all our data. We do this by first extracting the features for the acceleration data and gyroscope data seperately.

<b> Training data for acceleration features </b>

We load in all the filenames, then using map_dfr run our feature extraction function on all files and then bind the tables together row wise. Finally we rename the variables to indicate that they originate from accelartion data.

In [ ]:
# Acc features
filenames <- dir("./RawData/Train/", "^acc", full.names = TRUE)

# map_dfr runs `extractTimeDomainFeatures` on all elements in filenames 
# and binds results row wise
myData_acc <- map_dfr(filenames, extractDomainFeatures, sample_labels) 

# Label as Acc feats
myData_acc <- myData_acc %>% 
    rename_with(~paste0("AC_", .x), 6:length(names(myData_acc)))

# Check the result
print(myData_acc)

In [ ]:
# Check out distribution n_samples
myData_acc %>% group_by(AC_n_samples) %>% summarize(n = n())

In [ ]:
# There are some epochs which have fewer samples than 128
# We delete them to train our models only on those observations,
# which also provide a lot of data.

# The overall number of observations has been
myData_acc %>% nrow()

In [ ]:
# So, this exclusion is not drastic.
myData_acc <- myData_acc %>% filter(AC_n_samples == 128)
myData_acc %>% select(AC_n_samples) %>% pull() %>% unique() %>% print()

In [ ]:
# Finally, we remove this column, as its not a feature
myData_acc <- myData_acc %>% select(-AC_n_samples)

<b> Training data for gyroscope features </b>

We repeat the same process for the gyroscope data.

In [ ]:
# Gyro features
filenames <- dir("./RawData/Train/", "^gyro", full.names = TRUE)

# map_dfr runs `extractTimeDomainFeatures` on all elements in filenames 
# and binds results row wise
myData_gyro <- map_dfr(filenames, extractDomainFeatures, sample_labels) 

# Label as Acc feats
myData_gyro <- myData_gyro %>% 
    rename_with(~paste0("GY_", .x), 6:length(names(myData_gyro)))

# Check the result
print(myData_gyro)

In [ ]:
# Check out distribution n_samples
myData_gyro %>% group_by(GY_n_samples) %>% summarize(n = n())

In [ ]:
# There are some epochs which have fewer samples than 128
# We delete them to train our models only on those observations,
# which also provide a lot of data.

# The overall number of observations has been
myData_gyro %>% nrow()

In [ ]:
# So, this exclusion is not drastic.
myData_gyro <- myData_gyro %>% filter(GY_n_samples == 128)
myData_gyro %>% select(GY_n_samples) %>% pull() %>% unique() %>% print()

In [ ]:
# Finally, we remove this column, as its not a feature
myData_gyro <- myData_gyro %>% select(-GY_n_samples)

<b> Merging the accelaration and gyroscope features </b>

We merge our feature tables by left joining the accelaration features and gyroscope features by user id, activity label, sampleid and epoch.

In [ ]:
# Merge the two
train_df <- myData_acc %>% 
    left_join(myData_gyro,
              by = c("user_id", "exp_id", "activity", "sampleid", "epoch")
    )
# Check
train_df %>% select(c(1:8), c(85:87)) %>% head(3)

<b> Checking if all participants were correctly loaded in </b>

To make sure we loaded in the data of all participants and didn't leave anyone out we print the unique user IDs and check how many participants are present.

In [ ]:
# Check if other participants are also in there
train_df %>% 
   filter(user_id == 30) %>% 
   select(c(1:8), c(85:87)) %>% 
   head(3)

unique_users <- unique(train_df$user_id)
print(unique_users)
print(length(unique_users))

________________________________________________________________________________________________________

________________________________________________________________________________________________________
<h1 id=clean><span style="font-size: 20px;"><b> Data cleaning </h1></span></b>

However, our table is not ready for use just yet. We have to do some data cleaning before it's ready for use. Unlabelled epoch, near zero variation variables, highly correlated variables, highly correlated linear combinations are going to be removed.

<b> Unlabelled Epochs (i.e. no activity label) </b>

One problem with our data is that there are epochs without an activity label. We have to remove those before we fit our models.

In [ ]:
# How many observations per class?
train_df %>% group_by(activity) %>% summarize(n = n())

In [ ]:
# Remove unlabeled epochs
train_df <- train_df %>% filter(activity != "-")
train_df %>% group_by(activity) %>% summarize(n = n())
nrow(train_df)
ncol(train_df)

<b> Removing variables with near zero variation </b>

Some features might not be useable in some of our models because they have near zero variation. We remove variables with with near zero variation.

In [ ]:
## Near zero variation
nzv_var = caret::nearZeroVar(train_df[,6:ncol(train_df)])

<b> Removing multicollinearity (highly correlated variables) </b>

Variables that are highly correlated with others don't any predictive value to our model and can cause problems with fitting. We remove highly correlated variables.

In [ ]:
## Highly correlated variables
corr_var = train_df[,6:ncol(train_df)] %>%
    cor(use = "complete.obs") %>%
    caret::findCorrelation()

<b> Removing highly correlated linear combinations </b>

It can also be the case that a combination of variables in feature set are able to nearly perfectly predict other features in our data set. Keeping those variables does not add any value to our models. We remove highly correlated linear combinations.

Additionally, all these 3 checks are going to be saved in a vector ("to_be_removed")

In [ ]:
## Linear combinations
lc_var = train_df[,6:ncol(train_df)] %>%
    filter(complete.cases(.)) %>%
    caret::findLinearCombos()

# Save the columns of predictors to be removed
to_be_removed = c(nzv_var, corr_var, lc_var$remove) + 5
to_be_removed

In [ ]:
# Removing all indicated predictors detected above
train_df_3 = train_df[,-to_be_removed]

head(train_df_3)

dim(train_df_3)

Before feature selection, we also remove the 4 non-features that are left:

In [ ]:
train_df_3 <- train_df_3 %>% select(-c(epoch, user_id, exp_id, sampleid))
head(train_df_3)

________________________________________________________________________________________________________

________________________________________________________________________________________________________
<h1 id=fselect><span style="font-size: 20px;"><b> Feature selection </h1></span></b>

To prevent overfitting, we only want to use those features that are quite good at predicting the activities. We will therefore rank the features regarding their importance. Afterwards we will select the top X features (e.g. 10, 20, 30, 40, 50) and train our models on those feature subsets. Then we will do a kind of manual stepwise feature selection. The goal is to use as little features as possible while still maintaining a high accuracy.

<b> ANOVA Approach </b>

To rank the features regarding their importance we will run an ANOVA for each feature:
feature ~ activitiy
But we will do this in 5 subdataframes which each have a random 20% of the observations.
This way we get more reliabile estimates (F-values) about the importance for each features and whether this generalizes. After ranking the features in each of the 5 subdataframes regarding their F-values, we square the ranks to penalize lower rankings and then sum the squared rankings for each features across the 5 subdataframes. This yields the final ranking that we can then use for our stepwise inclusion during the model selection.

In [ ]:
# Factorize activity
train_df_3$activity <- factor(train_df_3$activity)

In [ ]:
# Define a function to calculate the F-value from an ANOVA
get_pred_fval <- function(predictor, activity) {
    # Perform an ANOVA with predictor and activity as factors
    current_aov <- aov(predictor ~ activity)
    
    # Summarize the ANOVA results
    result <- summary(current_aov)
    
    # Extract the F-value from the summary
    f_value <- result[[1]]$F
    f_value <- f_value[[1]]
    
    # Return the F-value
    return(f_value)
}

# Perform an ANOVA on the entire dataset (train_df_3$AC_sd3)
result <- aov(train_df_3$AC_sd3 ~ train_df_3$activity, data = train_df_3)

# Summarize the ANOVA results
result_summary <- summary(result)

# Extract the F-value from the summary
f_value <- result_summary[[1]]$F
f_value <- f_value[[1]]

# Print the F-value
f_value

# Print "test" as a marker
print("test")

# Call the get_pred_fval function for another predictor (train_df_3$AC_q1_25)
get_pred_fval(train_df_3$AC_q1_25, train_df_3$activity)

In [ ]:
# Create an empty vector to store F-values
f_values <- c()

# Loop through predictor columns (starting from the second column)
for (pred_index in 2:ncol(train_df_3)) {
    # Call the get_pred_fval function to calculate the F-value for each predictor
    current_f <- get_pred_fval(train_df_3[[pred_index]], train_df_3$activity)
    
    # Store the F-value in the f_values vector
    f_values[pred_index - 1] <- current_f
}

# Print the first few F-values (head)
head(f_values)

In [ ]:
# Get the column names of the original data frame (train_df_3)
predictor <- names(train_df_3)

# Exclude the first column (assumed to be the response variable)
predictor <- predictor[-1]

# Create a rank vector using square values to penalize lower rankings
rank <- seq(1:length(f_values)) ^ 2

# Create a new data frame df_concept
df_concept <- data.frame(
    predictor,   # Column names (predictor variables)
    f_values     # F-values calculated earlier
)

# Sort the data frame by F-values in descending order
sorted_df <- df_concept[order(-df_concept$f_values), ]

# Add a "rank" column to the sorted data frame
sorted_df$rank <- rank

# Select only the "predictor" and "rank" columns, excluding "f_values"
sorted_df <- sorted_df %>% select(-f_values)

# Print the first few rows of the sorted data frame
head(sorted_df)

Having outlined the procedure, we will now do create 5 subdataframes and do this in each of those.

In [ ]:
# First we shuffle train_df_3 by rows
set.seed(123)
# Get the number of rows in the dataframe
n_rows <- nrow(train_df_3)
# Create a random permutation of row indices
shuffled_indices <- sample(1:n_rows)
# Use the shuffled indices to reorder the rows of the dataframe
train_shuff <- train_df_3[shuffled_indices, ]

head(train_df_3, 3)
head(train_shuff, 3)

In [ ]:
# Split train_shuff into 5 smaller dataframes
n_rows <- nrow(train_shuff)
rows_per_split <- floor(n_rows / 5)  # Rows per split
split_dataframes <- list()

for (i in 1:5) {
  # Calculate the start and end row indices for the current split
  start_row <- (i - 1) * rows_per_split + 1
  end_row <- i * rows_per_split
  
  # Extract the rows for the current split
  current_split <- train_shuff[start_row:end_row, ]
  
  # Store the current split in the list
  split_dataframes[[i]] <- current_split
}

In [ ]:
# Peak into the splits
split_dataframes[[1]] %>% head(3)
split_dataframes[[2]] %>% head(3)

In [ ]:
# Function to create predictors ranked by Fvalue
ranked_predictors <- function(df) {
    
    # Calculate F-values
    f_values <- c()

    for (pred_index in 2:ncol(df)) {
        current_f = get_pred_fval(df[[pred_index]], df$activity)
        f_values[pred_index - 1] = current_f
    }
    
    # Ranked in new dataframe
    predictor <- names(train_df_3)
    predictor <- predictor[-1]
    rank <- seq(1:length(f_values)) ^ 2  # Square to penalize lower rankings

    ranked_df <- data.frame(
        predictor,
        f_values
    )

    sorted_df <- ranked_df[order(-ranked_df$f_values), ]
    sorted_df$rank = rank
    sorted_df <- sorted_df %>% select(-f_values)  # not important anymore
    
    return(sorted_df) 
}

In [ ]:
# Create the five rankings
ranked_1 <- ranked_predictors(split_dataframes[[1]])
ranked_2 <- ranked_predictors(split_dataframes[[2]])
ranked_3 <- ranked_predictors(split_dataframes[[3]])
ranked_4 <- ranked_predictors(split_dataframes[[4]])
ranked_5 <- ranked_predictors(split_dataframes[[5]])

head(ranked_1, 4)  # Note that the top 4 are different already
head(ranked_2, 4)

In [ ]:
# Join the 5 splits
total_ranking <- left_join(ranked_1, ranked_2, by = "predictor") %>%
                left_join(., ranked_3, by='predictor') %>%
                left_join(., ranked_4, by='predictor') %>%
                left_join(., ranked_4, by='predictor')
sample_n(total_ranking, 5)

In [ ]:
# Sum up
total_ranking <- total_ranking %>% mutate(rowsum = rowSums(select(total_ranking, 2:6)))

# arrange by rowsum
total_ranking <- total_ranking %>% 
   arrange(rowsum) %>%
   mutate(rank = seq(1,length(total_ranking$rowsum)))

head(total_ranking, 50)

# plot our feature 
total_ranking %>%
  ggplot(aes(x = rank, y = rowsum)) +
  geom_col()

<b> Select Top X features for Training </b>

In [ ]:
ranked_predictors <- total_ranking %>% select(predictor) %>% pull()  # as vector

train_df_top10 <- train_df_3 %>% select(activity, ranked_predictors[1:10])
train_df_top20 <- train_df_3 %>% select(activity, ranked_predictors[1:20])
train_df_top30 <- train_df_3 %>% select(activity, ranked_predictors[1:30])
train_df_top40 <- train_df_3 %>% select(activity, ranked_predictors[1:40])
train_df_top50 <- train_df_3 %>% select(activity, ranked_predictors[1:50])

________________________________________________________________________________________________________

________________________________________________________________________________________________________
<h1 id=modelfit><span style="font-size: 20px;"><b> Fitting the models </h1></span></b>

It's finnaly time to start fitting our models. We are going to fit the following types of models with all our lists of features (e.g. 10, 20, 30, 40, 50).

- Multinomal Logistic Regression
- K-nearest neighbour
- Linear Discriminant Analysis

Furthermore we are going to apply k-fold crossvalidation with k = 5 to to all our models to combat issues with overfitting but first we set.seed to be able to have consistent results.

In [ ]:
# Setting seed before model fitting
set.seed(1)

<b> Setting parameters for cross validation </b>

We set up parameters for useage in the cross validation.

In [ ]:
# Defining 5-fold cross validation (for all models)
trcntr <- trainControl('cv', number = 5, p = 0.8)

<b> Multinomal Logistic Regression with k-CV </b>

We fit our Multinomal Logistic Regressions.

In [ ]:
fit_mlr_10 <- caret::train(activity ~ ., 
                        data = train_df_top10, 
                        method = "multinom",
                        trace = FALSE,
                        trControl = trcntr)
fit_mlr_10

fit_mlr_20 <- caret::train(activity ~ ., 
                        data = train_df_top20, 
                        method = "multinom",
                        trace = FALSE,
                        trControl = trcntr)
fit_mlr_20

fit_mlr_30 <- caret::train(activity ~ ., 
                        data = train_df_top30, 
                        method = "multinom",
                        trace = FALSE,
                        trControl = trcntr)
fit_mlr_30

fit_mlr_40 <- caret::train(activity ~ ., 
                        data = train_df_top40, 
                        method = "multinom", 
                        trace = FALSE,
                        trControl = trcntr)
fit_mlr_40

fit_mlr_50 <- caret::train(activity ~ ., 
                        data = train_df_top50, 
                        method = "multinom", 
                        trace = FALSE,
                        trControl = trcntr)
fit_mlr_50

<b> k-NN with k-CV </b>

We fit our k-nearest neighbour models.

In [ ]:
fit_knn_10 <- caret::train(activity ~ ., 
                        data = train_df_top10,
                        preProcess = "scale", 
                        method = "knn",
                        trControl = trcntr)
fit_knn_10


fit_knn_20 <- caret::train(activity ~ ., 
                        data = train_df_top20,
                        preProcess = "scale", 
                        method = "knn",
                        trControl = trcntr)
fit_knn_20

fit_knn_30 <- caret::train(activity ~ ., 
                        data = train_df_top30,
                        preProcess = "scale", 
                        method = "knn",
                        trControl = trcntr)
fit_knn_30


fit_knn_40 <- caret::train(activity ~ ., 
                        data = train_df_top40,
                        preProcess = "scale", 
                        method = "knn",
                        trControl = trcntr)
fit_knn_40

fit_knn_50 <- caret::train(activity ~ ., 
                        data = train_df_top50,
                        preProcess = "scale", 
                        method = "knn",
                        trControl = trcntr)
fit_knn_50

<b> Linear Discriminant Analysis with k-CV </b>

We fit our linear discriminant analysis models.

In [ ]:
fit_lda_10 <- caret::train(activity ~ ., 
                       data = train_df_top10, 
                       method="lda", 
                       trControl = trcntr)
fit_lda_10


fit_lda_20 <- caret::train(activity ~ ., 
                       data = train_df_top20, 
                       method="lda", 
                       trControl = trcntr)
fit_lda_20

fit_lda_30 <- caret::train(activity ~ ., 
                       data = train_df_top30, 
                       method="lda", 
                       trControl = trcntr)
fit_lda_30


fit_lda_40 <- caret::train(activity ~ ., 
                       data = train_df_top40, 
                       method="lda", 
                       trControl = trcntr)
fit_lda_40

fit_lda_50 <- caret::train(activity ~ ., 
                       data = train_df_top50, 
                       method="lda", 
                       trControl = trcntr)
fit_lda_50

----------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------
<h1 id=compmodel><span style="font-size: 20px;"><b> Comparing models </h1></span></b>

We compare the peformance of the models we just fitted.

**First, we visualize performance with a barplot:**

In [ ]:
# All resulting models in a list
models <- list(
    MLR_10 = fit_mlr_10,
    MLR_20 = fit_mlr_20,
    MLR_30 = fit_mlr_30,
    MLR_40 = fit_mlr_40,
    MLR_50 = fit_mlr_50,
    
    kNN_10 = fit_knn_10,
    kNN_20 = fit_knn_20,
    kNN_30 = fit_knn_30,
    kNN_40 = fit_knn_40,
    kNN_50 = fit_knn_50,
    
    LDA_10 = fit_lda_10,
    LDA_20 = fit_lda_20,
    LDA_30 = fit_lda_30,
    LDA_40 = fit_lda_40,
    LDA_50 = fit_lda_50
)

# Extracting the cross-validated accuracies from each model
Acc <- sapply(models, function(mdl) max(mdl$results$Accuracy)) 

# Creating a barplot with only the best performing model in red
color <- 1 + (Acc >= max(Acc))
              
barplot(Acc, horiz = TRUE, las = 1, col = color, xlim = c(0, 1))
              
custom_ticks <- seq(0.1, 0.9, by = 0.1)
              
axis(1, at = custom_ticks, labels = custom_ticks)
              
best_model_index <- which.max(Acc)
              
abline(v = Acc[best_model_index], col = "red", lty = 2)

# Adding a title to the graph
title(main = "Comparing Model Accuracies")

# Determining the name of the most accurate model and its accuracy percentage
name_of_model <- names(models)[best_model_index]
accuracy_percentage <- round(Acc[best_model_index] * 100, 2)

# Creating a subtitle with the model name and accuracy percentage
subtitle <- sub <- sprintf("The most accurate model is %s with %.2f%% accuracy",
                           name_of_model, accuracy_percentage)

# Adding the subtitle to the graph
mtext(side = 3, text = subtitle, line = 0)
              
# Printing accuracy percentages
print(Acc*100)

**Second, we test if the performance differences between the models are statistically significant.**

For this reason we conduct a McNemar test (Chi-square version of the paired t-test, for contingency tables). We will use the pairwise.table function to conduct pairwise comparisons.

In [ ]:
## Test significance of classifier accuracy differences

# For each model in models compute a predicted label 
preds = sapply(models, predict)  # for each model, a columns of prediction 

# For each prediction, test if the prediction is correct
correct = (preds == train_df$activity) # matrix of TRUE and FALSE

# For pairwise.table() we need to define a function that returns the p-values
compare_func = function(i,j) mcnemar.test(correct[,i], correct[,j])$p.value

# Compute the table of pairwise comparisons
options(digits=3)
print(zapsmall(pairwise.table(compare_func, level.names = colnames(preds), "none")))

<b> Selecting the best model </b>

We select the best model for use in our predictions by hand out of the 15 models we have fitted
* LDA is generally behind kNN and MLR.

* kNN is flexible, but tend to struggle with higher dimensional data.

* kNN performs similarly to MLR at the top 40 and 50 features, and are statistically different (basd on the McNemar test) besides kNN_40 and MLR_50 (p = 0.085).

* MLR_50 performs slightly better than MLR_40 (89.59173% vs. 89.46948% respectively), but their performance is not significantly different (p = 0.383).

* On the other hand, MLR_40 performs almost a percent higher than MLR_30 (89.46948% vs. 88.36835%) and their performance is significantly different (p < 0.001)

**Therefore fit_mlr_40 is going to be selected as the model that we will use for further predictions.**

In [ ]:
winner_fit <- fit_mlr_40

----------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------
<h1 id=submissions><span style="font-size: 20px;"><b> Submissions </h1></span></b>


<b> Creating Dataframe for Test acceleration features </b>

In [ ]:
# Acc features
test_filenames <- dir("./RawData/Test/", "^acc", full.names = TRUE)
# map_dfr runs `extractTimeDomainFeatures` on all elements in filenames 
# and binds results row wise
test_myData_acc <- map_dfr(test_filenames, extractDomainFeatures, sample_labels) 

# Label as Acc feats
test_myData_acc <- test_myData_acc %>% rename_with(
    ~paste0("AC_", .x), 6:length(names(test_myData_acc))
)

# Check the result
print(test_myData_acc)

<b> Creating Dataframe for Test gyroscope features </b>

In [ ]:
# Gyro features
test_filenames <- dir("./RawData/Test/", "^gyro", full.names = TRUE)
test_myData_gyro <- map_dfr(test_filenames, extractDomainFeatures, sample_labels) 

# Label as Acc feats
test_myData_gyro <- test_myData_gyro %>% rename_with(
    ~paste0("GY_", .x), 6:length(names(test_myData_gyro))
)

# Check the result
print(test_myData_gyro)

<b> Merging acceleration and gyroscope test features </b>

In [ ]:
# Merge the two
test_df = test_myData_acc %>% 
    left_join(test_myData_gyro,
              by = c("user_id", "exp_id", "activity", "sampleid", "epoch")
    )
# Check
test_df %>% select(c(1:8), c(85:87)) %>% head(3)

In [ ]:
# Check if other participants are also in there
unique_users <- unique(test_df$user_id)
print(unique_users)
print(length(unique_users))

<b> Predicting Test Dataframe </b>

In [ ]:
# Predicting based on the winner model fit and training data
pred_df = predict(winner_fit, newdata = test_df, type = "prob")
pred_df %>% sample_n(5)

In [ ]:
# Adding column with highest probability in each row
pred_df$Predicted <- apply(pred_df, 1, function(row) {
  colnames(pred_df)[which.max(row)]
})

In [ ]:
# Checking if it works
pred_df %>% sample_n(5)

In [ ]:
# Attaching predictions to test_df
test_df = test_df %>% mutate(Predicted = pred_df$Predicted)

test_df %>% select(activity, Predicted) %>% head(5)  # Makes sense since no labels

In [ ]:
# Sanity check: lets view train_df
train_df %>% select(activity) %>% head(3)

<b> Formatting the submission file </b>

In [ ]:
test_df %>%

    # prepend "user" and "exp" to user_id and exp_id
    mutate(
        user_id = paste(ifelse(user_id < 10, "user0", "user"), user_id, sep=""), 
        exp_id = paste(ifelse(exp_id < 10, "exp0", "exp"), exp_id, sep="")
    ) %>% 

    # unit columnes user_id, exp_id and sample_id into a string 
    # separated by "_" and store it in the new variable `Id`
    unite(Id, user_id, exp_id, sampleid) %>%

    # retain only the `Id` and  predictions
    select(Id, Predicted = Predicted) %>%

    # write to file
    write_csv("test_set_predictions.csv")


# Check the result: print first 20 lines in the submission file
cat(readLines("test_set_predictions.csv", 6), sep="\n")

----------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------
<h1 id=author><span style="font-size: 20px;"><b> Author Contributions </h1></span></b>

Bence: timedom features, spectral features, model fitting, textual edits, overall feature set creation, base feature selection, cleaning up workbook.

Jeroen: code styling, html styling, writing of the notebook, found and fixed bug that caused our models to be unable to fit, corrected coding mistake in ANOVA procedure, plotted ranked sum. Committed feature theft.

Vincent: cleaning workbook, import gyro data, train_df, test_df, making predictions, spectral features from group 16, removed epochs with < 128 samples, implemented ANOVA feature selection

----------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------
<h1 id=reference><span style="font-size: 20px;"><b> References </h1></span></b>

* dan_vdmeer, Dave Leitritz, Joost van Kordelaar, Raoul. (2023). BDA 2023 Physical activity recognition. Kaggle. https://kaggle.com/competitions/physical-activity-recognition-bda-2023
* Grasman, R. (2018). Feature extraction from Signals. Dropbox Paper. Retrieved September 21, 2023, from https://paper.dropbox.com/doc/Feature-extraction-from-Signals-qCp5uvj47gmyuw5nmB8lL

----------------------------------------------------------------------------------------------------------------------------